This is stripped down to keep code as simple as possible for running
the SSD1306 OLED attached to the TTGO ESP32 through I2C


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [2]:
from machine import I2C, Pin

rst = Pin(16, Pin.OUT)
rst.value(1)
i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=450000)

i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5)) # for the ESP8266

print("i2c devices", i2c.scan(), "must contain", 0x3c)


i2c devices [60] must contain 60


In [3]:
import framebuf

# There is an extra byte to the data buffer to hold an I2C data/command byte
# to use hardware-compatible I2C transactions.
buffer = bytearray(((64 // 8) * 128) + 1)
buffer[0] = 0x40  # Set first byte of data buffer to Co=0, D/C=1
fbuff = framebuf.FrameBuffer1(memoryview(buffer)[1:], 128, 64)

cmdforinit = bytes((0xae,        # CMD_DISP=off
                    0x20, 0x00,  # SET_MEM_ADDR  horizontal
                    0x40,        # SET_DISP_START_LINE
                    0xa0 | 0x01, # column addr 127 mapped to SEG0
                    0xa8, 63,    # SET_MUX_RATIO, height-1
                    0xc0 | 0x08, # SET_COM_OUT_DIR scan from COM[N] to COM0
                    0xd3, 0x00,  # SET_DISP_OFFSET
                    0xda, 0x12,  # SET_COM_PIN_CFG
                    0xd5, 0x80,  # SET_DISP_CLK_DIV
                    0xd9, 0xf1,  # SET_PRECHARGE
                    0xdb, 0x30,  # SET_VCOM_DESEL 0.83*Vcc
                    0x81, 0xff,  # SET_CONTRAST maximum
                    0xa4,        # SET_ENTIRE_ON output follows RAM contents
                    0xa6,        # SET_NORM_INV not inverted
                    0x8d, 0x14,  # SET_CHARGE_PUMP
                    0xae | 0x01  # SET_DISP on
                   ))
for c in cmdforinit:
    i2c.writeto(0x3c, bytes((0x80, c)))

cmdforshow = bytes((0x80, 0x21, # SET_COL_ADDR
                    0x80, 0,    # 0
                    0x80, 127,  # width-1
                    0x80, 0x22, # SET_PAGE_ADDR
                    0x80, 0,    # 0
                    0x80, 7     # height//8 - 1
                    ))   
 
def oledshow():
    i2c.writeto(0x3c, cmdforshow)
    i2c.writeto(0x3c, buffer)
    
def oledcontrast(contrast):
    i2c.writeto(0x3c, bytes((0x80, 0x81, 0x80, contrast)))

def oledinvert(invert=True):
    i2c.writeto(0x3c, bytes((0x80, 0xa6 | (invert & 1))))



In [9]:
help(fbuff)

object <FrameBuffer> is of type FrameBuffer
  fill -- <function>
  fill_rect -- <function>
  pixel -- <function>
  hline -- <function>
  vline -- <function>
  rect -- <function>
  line -- <function>
  blit -- <function>
  scroll -- <function>
  text -- <function>


In [4]:
fbuff.rect(9, 9, 90, 30, 1)
oledshow()

In [5]:
fbuff.fill(0)
fbuff.text("assaZZZZ", 10, 10, 1)
oledshow()


In [11]:
# checkerboard
for i in range(0, 128, 8):
    for j in range(0, 64, 8):
        fbuff.fill_rect(i, j, 8, 8, (i//8 + j//8)%2)
oledshow()

In [14]:
import os
print(os.listdir())

['boot.py']


In [77]:
%%writefile OLED_driver.py

from machine import I2C, Pin

rst = Pin(16, Pin.OUT)
rst.value(1)
i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=450000)
assert 0x3c in i2c.scan(), "OLED i2c not found"

import framebuf

# There is an extra byte to the data buffer to hold an I2C data/command byte
# to use hardware-compatible I2C transactions.
buffer = bytearray(((64 // 8) * 128) + 1)
buffer[0] = 0x40  # Set first byte of data buffer to Co=0, D/C=1
fbuff = framebuf.FrameBuffer1(memoryview(buffer)[1:], 128, 64)

cmdforinit = bytes((0xae,        # CMD_DISP=off
                    0x20, 0x00,  # SET_MEM_ADDR  horizontal
                    0x40,        # SET_DISP_START_LINE
                    0xa0 | 0x01, # column addr 127 mapped to SEG0
                    0xa8, 63,    # SET_MUX_RATIO, height-1
                    0xc0 | 0x08, # SET_COM_OUT_DIR scan from COM[N] to COM0
                    0xd3, 0x00,  # SET_DISP_OFFSET
                    0xda, 0x12,  # SET_COM_PIN_CFG
                    0xd5, 0x80,  # SET_DISP_CLK_DIV
                    0xd9, 0xf1,  # SET_PRECHARGE
                    0xdb, 0x30,  # SET_VCOM_DESEL 0.83*Vcc
                    0x81, 0xff,  # SET_CONTRAST maximum
                    0xa4,        # SET_ENTIRE_ON output follows RAM contents
                    0xa6,        # SET_NORM_INV not inverted
                    0x8d, 0x14,  # SET_CHARGE_PUMP
                    0xae | 0x01  # SET_DISP on
                   ))
for c in cmdforinit:
    i2c.writeto(0x3c, bytes((0x80, c)))

cmdforshow = bytes((0x80, 0x21, # SET_COL_ADDR
                    0x80, 0,    # 0
                    0x80, 127,  # width-1
                    0x80, 0x22, # SET_PAGE_ADDR
                    0x80, 0,    # 0
                    0x80, 7     # height//8 - 1
                    ))   
 
def oledshow():
    i2c.writeto(0x3c, cmdforshow)
    i2c.writeto(0x3c, buffer)
    
def oledcontrast(contrast):
    i2c.writeto(0x3c, bytes((0x80, 0x81, 0x80, contrast)))

def oledinvert(invert=True):
    i2c.writeto(0x3c, bytes((0x80, 0xa6 | (invert & 1))))

# checkerboard starting page
for i in range(0, 128, 8):
    for j in range(0, 64, 8):
        fbuff.fill_rect(i, j, 8, 8, (i//8 + j//8)%2)
oledshow()

Writing OLED_driver.py



In [79]:
%sendtofile --source OLED_driver.py

Sent 58 lines (2216 bytes) to OLED_driver.py.


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [11]:
from OLED_driver import i2c, fbuff, oledshow

In [12]:
fbuff.fill(0)
fbuff.text("1234", 10, 10, 1)
def doublepixels():
    for x in range(63, -1, -1):
        for y in range(31, -1, -1):
            fbuff.fill_rect(x*2,y*2,2,2,fbuff.pixel(x,y))
doublepixels()
oledshow()


In [13]:
fbuff.fill(0)
fbuff.text("1234", 10, 10, 1)

# chr45=- . / 0 1 2 3 4 5 6 7 8 9
num8x8 = b'\x00\x08\x08\x08\x08\x08\x08\x00\x00\x00\x00``\x00\x00\x00\x00@`0\x18\x0c\x06\x02\x00>\x7fIE\x7f>\x00\x00@D\x7f\x7f@@\x00\x00bsQIOF\x00\x00"cII\x7f6\x00\x00\x18\x18\x14\x16\x7f\x7f\x10\x00\'gEE}9\x00\x00>\x7fII{2\x00\x00\x03\x03y}\x07\x03\x00\x006\x7fII\x7f6\x00\x00&oII\x7f>\x00'
def fatntext(t, x, y):  # numbers only
    for s in t:
        k = max(0, min(11, ord(s) - 45))
        for c in num8x8[k*8:k*8+8]:
            for j in range(8):
                if (c & (1<<j)):
                    fbuff.fill_rect(x,y+j*2,2,2,1)
            x += 2
fatntext("-88.19", 20, 20)
oledshow()


In [ ]:
font_petme128_8x8.h

0x00,0x08,0x08,0x08,0x08,0x08,0x08,0x00, // 45=-
0x00,0x00,0x00,0x60,0x60,0x00,0x00,0x00, // 46=.
    0x00,0x40,0x60,0x30,0x18,0x0c,0x06,0x02, // 47=/
    0x00,0x3e,0x7f,0x49,0x45,0x7f,0x3e,0x00, // 48=0
    0x00,0x40,0x44,0x7f,0x7f,0x40,0x40,0x00, // 49=1
    0x00,0x62,0x73,0x51,0x49,0x4f,0x46,0x00, // 50=2
    0x00,0x22,0x63,0x49,0x49,0x7f,0x36,0x00, // 51=3
    0x00,0x18,0x18,0x14,0x16,0x7f,0x7f,0x10, // 52=4
    0x00,0x27,0x67,0x45,0x45,0x7d,0x39,0x00, // 53=5
    0x00,0x3e,0x7f,0x49,0x49,0x7b,0x32,0x00, // 54=6
    0x00,0x03,0x03,0x79,0x7d,0x07,0x03,0x00, // 55=7
    0x00,0x36,0x7f,0x49,0x49,0x7f,0x36,0x00, // 56=8
    0x00,0x26,0x6f,0x49,0x49,0x7f,0x3e,0x00, // 57=9


In [158]:

%sendtofile --quiet --source OLED_grapher.py


Sent 144 lines (4068 bytes) to OLED_grapher.py.


In [161]:
from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph, scrollgraphtest

[leftinbuffer] ['OK\x04\x04>']


In [162]:
scrollgraphtest(fbuff, oledshow)

....

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "OLED_grapher.py", line 143, in scrollgraphtest
KeyboardInterrupt: 


In [165]:
import math, time
scrollinit()
for i in range(0, 10000, 10):
    addscrollgraph(math.sin(math.radians(i))*200+i/2, i*10)
    plotscrollgraph(fbuff)
    oledshow()


.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 7, in <module>
KeyboardInterrupt: 


In [159]:
%rebootdevice

normal repl mode not detected b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\n'
not command mode
[Timed out waiting for recognizable response]
